In [1]:
import pandas as pd 
import numpy as np
import pyarrow.parquet as pq

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from time import time

## Q1. Downloading the data

In [2]:
file1 = pq.ParquetFile('./data/yellow_tripdata_2023-01.parquet')
file2 = pq.ParquetFile('./data/yellow_tripdata_2023-02.parquet')

table1 = file1.read()
table2 = file2.read()

df_jan = table1.to_pandas()
df_feb = table2.to_pandas()

In [3]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

There are 19 columns

## Q2. Computing duration

In [4]:
# For January
df_jan['duration'] = (df_jan['tpep_dropoff_datetime']) - (df_jan['tpep_pickup_datetime'])
df_jan['duration'] = np.round((df_jan['duration']) / np.timedelta64(1, 'm'), decimals=2)

# For February
df_feb['duration'] = (df_feb['tpep_dropoff_datetime']) - (df_feb['tpep_pickup_datetime'])
df_feb['duration'] = np.round((df_feb['duration']) / np.timedelta64(1, 'm'), decimals=2)

In [6]:
df_jan.describe().round(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3066766.000,3066766,3066766,2995023.000,3066766.000,2995023.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,2995023.000,2995023.000,3066766.000
mean,1.730,2023-01-17 00:22:26.288164,2023-01-17 00:38:06.427874,1.363,3.847,1.497,166.398,164.393,1.194,18.367,1.538,0.488,3.368,0.518,0.982,27.020,2.274,0.107,15.669
min,1.000,2008-12-31 23:01:42,2009-01-01 14:29:11,0.000,0.000,1.000,1.000,1.000,0.000,-900.000,-7.500,-0.500,-96.220,-65.000,-1.000,-751.000,-2.500,-1.250,-29.200
25%,1.000,2023-01-09 16:21:57.250000,2023-01-09 16:37:06,1.000,1.060,1.000,132.000,114.000,1.000,8.600,0.000,0.500,1.000,0.000,1.000,15.400,2.500,0.000,7.120
50%,2.000,2023-01-17 08:42:29.500000,2023-01-17 08:58:30.500000,1.000,1.800,1.000,162.000,162.000,1.000,12.800,1.000,0.500,2.720,0.000,1.000,20.160,2.500,0.000,11.520
75%,2.000,2023-01-24 16:26:27,2023-01-24 16:42:49,1.000,3.330,1.000,234.000,234.000,1.000,20.500,2.500,0.500,4.200,0.000,1.000,28.700,2.500,0.000,18.300
max,2.000,2023-02-01 00:56:53,2023-02-02 09:28:47,9.000,258928.150,99.000,265.000,265.000,4.000,1160.100,12.500,53.160,380.800,196.990,1.000,1169.400,2.500,1.250,10029.180
std,0.444,NaN,NaN,0.896,249.584,6.475,64.244,69.944,0.529,17.808,1.790,0.103,3.827,2.018,0.183,22.164,0.772,0.356,42.594


The standard deviation of the trips duration in January is 42.59

## Q3. Dropping outliers

In [7]:
# For January
df_jan = df_jan[(df_jan['duration'] >= 1.00) & (df_jan['duration'] <= 60.00)]

# For February
df_feb = df_feb[(df_feb['duration'] >= 1.00) & (df_feb['duration'] <= 60.00)]

In [9]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

#### Calculation of the record's fraction left

In [10]:
100 - (((3066766 - 3009173) * 100) / 3066766)

98.1220282212598

The fraction of the records left after dropping outliers is 98%

## Q4. One-hot encoding

In [17]:
train_feat = df_jan[['PULocationID', 'DOLocationID']].astype(str)
train_dicts = train_feat.to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [18]:
X_train.shape

(3009173, 515)

The dimensionality of this matrix is 515

## Q5. Training a model

In [19]:
y_train = df_jan['duration'].values

In [20]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

c:\Users\Admin\Documents\a_data_workspace\datatalks\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261720998863

The RMSE on train is 7.64

## Q6. Evaluating the model

In [21]:
#One-Hot Encoding
val_feat = df_feb[['PULocationID', 'DOLocationID']].astype(str)
val_dicts = val_feat.to_dict(orient='records')

X_val = dv.transform(val_dicts)

In [22]:
# Prediction on val
y_val = df_feb['duration'].values

y_val_pred = lr.predict(X_val)

mean_squared_error(y_val, y_val_pred, squared=False)

The RMSE on validation is 7.81